In [ ]:
import numpy as np
from subprocess import PIPE, run
import matplotlib.pyplot as plt
import os
import textwrap

class ExptBuilder():
    def __init__(self):
        self.__code_path__ = os.environ.get('code')
        self.__temp_exp_path__ = os.path.join(self.__code_path__, "k-exp", "kexp", "experiments", "ml_expt.py")

    def run_expt(self):
        expt_path = self.__temp_exp_path__
        run_expt_command = r"%kpy% & artiq_run " + expt_path
        result = run(run_expt_command, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
        print(result.returncode, result.stdout, result.stderr)
        os.remove(self.__temp_exp_path__)
        return result.returncode
    
    def write_experiment_to_file(self, program):
        with open(self.__temp_exp_path__, 'w') as file:
            file.write(program)
    
    def fringe_scan_expt(self, amp_img, freq_raman):
        script = textwrap.dedent(f"""
        from artiq.experiment import *
        from artiq.experiment import delay
        from kexp import Base, img_types, cameras
        import numpy as np
        from kexp.calibrations.tweezer import tweezer_vpd1_to_vpd2
        from kexp.calibrations.imaging import high_field_imaging_detuning
        from artiq.coredevice.sampler import Sampler
        from artiq.language import now_mu

        class cont_mon_182_ref(EnvExperiment, Base):

            def prepare(self):
                Base.__init__(self,setup_camera=True,
                            camera_select=cameras.andor,
                            save_data=True,
                            imaging_type=img_types.DISPERSIVE)

                # self.p.v_pd_hf_tweezer_1064_rampdown2_end = 1.
                
                # self.xvar('beans',[0,1]*50)

                self.p.i_hf_raman = 182.

                # self.xvar('frequency_raman_transition',147.35e6 + np.linspace(-30.e3,30.e3,9))

                # self.p.frequency_raman_transition = 145.57e6 # 191. A
                # self.p.frequency_raman_transition = 147.2447e6 # 182. A
                self.p.frequency_raman_transition = {freq_raman}

                # self.xvar('amp_raman',np.linspace(0.1,.35,15))
                self.p.fraction_power_raman = .99

                # self.xvar('t_raman_stateprep_pulse',[0.,9.9979e-06])
                self.p.t_raman_stateprep_pulse = (1.0058e-05) / 2

                # self.xvar('t_continuous_rabi',np.linspace(0.,400.e-6,10))
                self.p.t_continuous_rabi = 200.e-6
                
                # self.xvar('amp_imaging',np.linspace(0.3,.6,25))
                # self.p.amp_imaging = .28
                self.p.amp_imaging = {amp_img}

                self.p.hf_imaging_detuning = -565.e6 # 182.

                # self.xvar('hf_imaging_detuning_mid',np.arange(-710.,-570.,10)*1.e6)
                # self.p.hf_imaging_detuning_mid = -690.e6 # -635.e6
                self.p.hf_imaging_detuning_mid = -650.e6 # -635.e6
                
                # self.xvar('dimension_slm_mask',np.linspace(10.e-6,100.e-6,10))
                # self.p.dimension_slm_mask = 60.e-6
                # self.xvar('phase_slm_mask',np.linspace(0.,2.7*np.pi,15))
                self.p.phase_slm_mask = 1.54 * np.pi
                self.p.dimension_slm_mask = 20.e-6

                # self.xvar('t_raman_stateprep_pulse',[0.e-6,29.e-6]*50)

                # self.xvar('t_tweezer_hold',np.linspace(1.e-3,1.1e-3,10))
                self.p.t_tweezer_hold = 20.e-3
                self.p.t_tof = 20.e-6
                self.p.t_mot_load = 1.15

                # self.camera_params.exposure_time = 20.e-6
                # self.params.t_imaging_pulse = self.camera_params.exposure_time
                
                self.p.N_repeats = 10

                self.scope = self.scope_data.add_siglent_scope("192.168.1.108", label='PD', arm=False)

                self.finish_prepare(shuffle=True)

            @kernel
            def scan_kernel(self):

                self.set_imaging_detuning(frequency_detuned = self.p.hf_imaging_detuning_mid)
                # self.set_imaging_detuning(frequency_detuned = self.p.hf_imaging_detuning)
                self.slm.write_phase_mask_kernel(phase=self.p.phase_slm_mask)
                self.imaging.set_power(self.p.amp_imaging)

                self.prepare_hf_tweezers()

                self.raman.init(fraction_power = self.p.fraction_power_raman,
                                frequency_transition = self.p.frequency_raman_transition)

                self.ttl.raman_shutter.on()
                delay(10.e-3)
                self.ttl.line_trigger.wait_for_line_trigger()
                delay(5.7e-3)

                # self.raman.pulse(t=self.p.t_raman_stateprep_pulse)

                self.imaging.on()
                self.ttl.pd_scope_trig3.pulse(1.e-6)
                self.raman.on()
                delay(self.p.t_continuous_rabi)
                self.raman.off()
                self.imaging.off()

                self.ttl.raman_shutter.off()
                
                self.set_high_field_imaging(self.p.i_hf_raman)
                self.imaging.set_power(.25,reset_pid=True)

                delay(self.p.t_tweezer_hold)
                self.tweezer.off()

                delay(self.p.t_tof)

                self.abs_image()

                self.core.wait_until_mu(now_mu())
                self.scope.read_sweep(0)
                self.core.break_realtime()
                delay(30.e-3)

            @kernel
            def run(self):
                self.init_kernel()
                self.load_2D_mot(self.p.t_2D_mot_load_delay)
                self.scan()
                self.mot_observe()

            def analyze(self):
                import os
                expt_filepath = os.path.abspath(__file__)
                self.end(expt_filepath)
                """)
        return script

In [2]:
eBuilder = ExptBuilder()

In [ ]:
img_amp = np.linspace(.1,.9,20)
raman_freq = [1.47259750e+08, 1.47265961e+08, 1.47272171e+08, 1.47278382e+08,
       1.47284592e+08, 1.47290803e+08, 1.47297013e+08, 1.47303224e+08,
       1.47309434e+08, 1.47315645e+08, 1.47321855e+08, 1.47328066e+08,
       1.47334276e+08, 1.47340487e+08, 1.47346697e+08, 1.47352908e+08,
       1.47359118e+08, 1.47365329e+08, 1.47371539e+08, 1.47377750e+08]
# np.random.shuffle(ts)
for f in range(len(raman_freq)):
    print(img_amp[f],raman_freq[f])
    eBuilder.write_experiment_to_file(eBuilder.fringe_scan_expt(amp_img=img_amp[f],freq_raman=raman_freq[f]))
    eBuilder.run_expt()

0.1 147260750.0
0 Run id: 57209
 10 values of dummy. 10 total shots. 30 total images expected.
Acknowledged camera ready signal.
Camera is ready.

Sent: {'mask': 'spot', 'center': [1057, 808], 'phase': 0.39, 'dimension': 20, 'initialize': False, 'spacing': 10, 'angle': 45}
-> mask: spot, dimension = 20 um, phase = 0.39 pi, x-center = 1057, y-center = 808

 Run ID: 57209

Sent: {'mask': 'spot', 'center': [1057, 808], 'phase': 0.39, 'dimension': 20, 'initialize': False, 'spacing': 10, 'angle': 45}
-> mask: spot, dimension = 20 um, phase = 0.39 pi, x-center = 1057, y-center = 808


Sent: {'mask': 'spot', 'center': [1057, 808], 'phase': 0.39, 'dimension': 20, 'initialize': False, 'spacing': 10, 'angle': 45}
-> mask: spot, dimension = 20 um, phase = 0.39 pi, x-center = 1057, y-center = 808


Sent: {'mask': 'spot', 'center': [1057, 808], 'phase': 0.39, 'dimension': 20, 'initialize': False, 'spacing': 10, 'angle': 45}
-> mask: spot, dimension = 20 um, phase = 0.39 pi, x-center = 1057, y-cente

In [4]:
from kexp.control.relay_control import EthernetRelay
relay = EthernetRelay()

In [5]:
relay.source_off()